- loading data

In [66]:
! wget https://raw.githubusercontent.com/mohammedashrafdagga/ds_lr/main/dataset/melb_data.csv

--2022-01-24 09:36:11--  https://raw.githubusercontent.com/mohammedashrafdagga/ds_lr/main/dataset/melb_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2091239 (2.0M) [text/plain]
Saving to: ‘melb_data.csv.2’

melb_data.csv.2     100%[===================>]   1.99M  --.-KB/s    in 0.08s   

2022-01-24 09:36:11 (26.3 MB/s) - ‘melb_data.csv.2’ saved [2091239/2091239]



-- reading and importing libaray

In [67]:
# improt 
import pandas as pd

data_melb = pd.read_csv('melb_data.csv')

# looking a head of data
data_melb.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [68]:
data_melb.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

-- definte X and y

In [69]:
y = data_melb['Price']
X = data_melb.drop('Price', axis = 1)

In [70]:
X.head()

,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [71]:
# split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1, test_size = 0.2)

In [72]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10864, 20)
(2716, 20)
(10864,)
(2716,)


-- How Know if have missing value or not

In [73]:
# isnull()
X.isnull().any()

Suburb           False
Address          False
Rooms            False
Type             False
Method           False
SellerG          False
Date             False
Distance         False
Postcode         False
Bedroom2         False
Bathroom         False
Car               True
Landsize         False
BuildingArea      True
YearBuilt         True
CouncilArea       True
Lattitude        False
Longtitude       False
Regionname       False
Propertycount    False
dtype: bool

In [74]:
# return just columns with missing value
col_with_missing_value = [col for col in X.columns if X[col].isnull().sum() > 0]
col_with_missing_value

['Car', 'BuildingArea', 'YearBuilt', 'CouncilArea']

## Approaches working with Missing Value

#### 1. drop columns containing missing value

In [75]:
# is better make copy of data
X_train_missing = X_train.copy()
X_test_missing = X_test.copy()

before drop columns containing missing value

In [76]:
print(X_train_missing.shape)
print(X_test_missing.shape)

(10864, 20)
(2716, 20)


In [77]:
X_train_missing = X_train_missing.drop(col_with_missing_value, axis = 1)
X_test_missing = X_test_missing.drop(col_with_missing_value, axis = 1)

After drop columns containing missing value

In [78]:
print(X_train_missing.shape)
print(X_test_missing.shape)

(10864, 16)
(2716, 16)


In [79]:
len(X_train_missing.columns)

16

# now make Label encoder to preprocessing Category Col

In [81]:
# now precess data
from sklearn.preprocessing import OneHotEncoder

# Make copy to avoid changing original data 
OH_cols_train =  X_train_missing.copy()
OH_cols_test = X_test_missing.copy()

# Apply ordinal encoder to each column with categorical data
oh = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(oh.fit_transform(X_train_missing[col_object]))
OH_cols_test = pd.DataFrame(oh.transform(X_test_missing[col_object]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train_missing.index
OH_cols_test.index = X_test_missing.index


num_X_train = label_X_train.drop(col_object, axis=1)
num_X_test = label_X_test.drop(col_object, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_test, OH_cols_test], axis=1)

In [82]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches

model = RandomForestRegressor(n_estimators=10, random_state=0)
model.fit(OH_X_train, y_train)
preds = model.predict(OH_X_valid)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


#### Calaluate MAE after apply Approaches 1

In [84]:
print("MAE is: ")
print(mean_absolute_error(y_test, preds))

MAE is: 
168194.1248527246
